In [ ]:
import os
import uuid
from typing import List, Tuple, Literal, Annotated

from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, ToolMessage
from langchain_core.pydantic_v1 import BaseModel
from pydantic import BaseModel
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
import gradio as gr

from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client("gen-ai-hub")

import os
import json
from dotenv import load_dotenv, find_dotenv

load_dotenv()

e:\LangGraph_Agents\myenv\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
e:\LangGraph_Agents\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
aicore_home = os.getenv("AICORE_HOME")
profile = os.getenv("AICORE_PROFILE", "default")
config_path = os.path.join(aicore_home, f"config.json")
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# 구성 파일에서 값 읽어오기
with open(config_path, "r") as config_file:
    config = json.load(config_file)

# 클라이언트 초기화
ai_core_client = AICoreV2Client(
    base_url=config["AICORE_BASE_URL"],
    auth_url=config["AICORE_AUTH_URL"],
    client_id=config["AICORE_CLIENT_ID"],
    client_secret=config["AICORE_CLIENT_SECRET"],
)

print("AI Core client initialized successfully!")

AI Core client initialized successfully!


In [5]:
from langchain_core.prompts import ChatPromptTemplate


from langchain_openai import ChatOpenAI

# Assume OPENAI_API_KEY is set
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()

# This creates a RunnableSequence instance
chain = prompt | model

In [7]:
response = chain.invoke({"topic": "cats"})
print(response.content)

Why was the cat sitting on the computer?

Because it wanted to keep an eye on the mouse!


In [ ]:
from langchain_core.runnables import RunnableParallel

# Example setup (replace with actual runnables)
runnable1 = ChatPromptTemplate.from_template("Runnable 1: {input}") | model
runnable2 = ChatPromptTemplate.from_template("Runnable 2: {input}") | model

# Define parallel execution using a dictionary
parallel_chain = RunnableParallel(
    steps={
        "output1": runnable1,
        "output2": runnable2,
    }
)

# Or equivalently:
# parallel_chain = {"output1": runnable1, "output2": runnable2}

# Invoking this runs runnable1 and runnable2 potentially in parallel
# The input is passed to *both* runnables
output = parallel_chain.invoke({"input": "parallel processing"})

# Output will be a dictionary:
# {'output1': AIMessage(...), 'output2': AIMessage(...)}

In [22]:
pprint(output)

{'steps': {'output1': AIMessage(content='Parallel processing is a method of executing multiple tasks simultaneously, speeding up data processing and computation. This can be achieved by splitting a large task into smaller sub-tasks and executing them in parallel on multiple processors or cores. \n\nIn this runnable example, I will demonstrate parallel processing using the Java programming language. We will create a simple program that calculates the square of numbers in parallel.\n\n```java\nimport java.util.concurrent.ExecutorService;\nimport java.util.concurrent.Executors;\n\npublic class ParallelProcessingExample {\n\n    public static void main(String[] args) {\n        int[] numbers = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};\n\n        ExecutorService executor = Executors.newFixedThreadPool(5);\n\n        for (int number : numbers) {\n            executor.submit(() -> {\n                int square = number * number;\n                System.out.println("Square of " + number + " is: " + squ

In [23]:
pprint(output["steps"]["output1"].content)

('Parallel processing is a method of executing multiple tasks simultaneously, '
 'speeding up data processing and computation. This can be achieved by '
 'splitting a large task into smaller sub-tasks and executing them in parallel '
 'on multiple processors or cores. \n'
 '\n'
 'In this runnable example, I will demonstrate parallel processing using the '
 'Java programming language. We will create a simple program that calculates '
 'the square of numbers in parallel.\n'
 '\n'
 '```java\n'
 'import java.util.concurrent.ExecutorService;\n'
 'import java.util.concurrent.Executors;\n'
 '\n'
 'public class ParallelProcessingExample {\n'
 '\n'
 '    public static void main(String[] args) {\n'
 '        int[] numbers = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};\n'
 '\n'
 '        ExecutorService executor = Executors.newFixedThreadPool(5);\n'
 '\n'
 '        for (int number : numbers) {\n'
 '            executor.submit(() -> {\n'
 '                int square = number * number;\n'
 '                Sys

In [ ]:
# Example of passing configuration
result = chain.with_config({"run_name": "JokeGenerationRun"}).invoke(
    {"topic": "robots"}
)

In [26]:
from pprint import pprint

In [25]:
pprint(result)  # Should print "JokeGenerationRun"

AIMessage(content='Why did the robot go on a diet? Because it had too many bytes!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 13, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrDO8648dDD9qoqqhoibzdVIoj7js', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c2a60af9-35cc-4027-a3f1-463e3e85aa0e-0', usage_metadata={'input_tokens': 13, 'output_tokens': 16, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


In [27]:
# Synchronous Example
# Assume setup for prompt, model, parser is done
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()
sync_chain = prompt | model | output_parser

result = sync_chain.invoke({"topic": "programmers"})
print(result)

Why do programmers prefer dark mode? Because light attracts bugs!


In [ ]:
import asyncio

# Assume setup for prompt, model, parser is done (as above)
# sync_chain = prompt | model | output_parser # LCEL chains automatically support async if components do


async def run_async_chain():
    print("Running async chain...")
    # Use ainvoke instead of invoke
    result = await sync_chain.ainvoke({"topic": "data scientists"})
    print(result)
    # If model supports streaming:
    # print("Streaming response:")
    # async for chunk in sync_chain.astream({"topic": "async programming"}):
    #   print(chunk, end="", flush=True)
    # print()


async def main():
    await run_async_chain()


if __name__ == "__main__":
    # Start the asyncio event loop
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Assume 'llm' is an initialized ChatOpenAI instance
llm = ChatOpenAI()
# Step 1: Initial processing, maybe extract entities
prompt1 = ChatPromptTemplate.from_template("Extract names from: {input}")
chain1 = prompt1 | llm

# Step 2: Use extracted names (state) along with original input for next step
prompt2 = ChatPromptTemplate.from_template(
    "Generate a greeting for {name} based on this context: {original_input}"
)
chain2 = prompt2 | llm

# Combine, passing original input and adding 'name' to the state dict
# The input to this chain is expected to be a dictionary, e.g., {"input": "John Doe visited Paris."}
complex_chain = (
    RunnablePassthrough.assign(
        name=chain1,  # Runs chain1, adds result under 'name' key
        original_input=lambda x: x["input"],  # Passes 'input' key as 'original_input'
    )
    | chain2
)  # chain2 now receives {'name': 'John Doe', 'original_input': '...'}

# Example invocation:
result = complex_chain.invoke({"input": "Alice went to the store."})
print(result)

content='Hello, Alice! How was your trip to the store?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 277, 'total_tokens': 289, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrE90HD5kmHWjWKhTIaCk9ZyRpGXd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--c5e9f3bf-f6a1-4124-b49f-62908337d00d-0' usage_metadata={'input_tokens': 277, 'output_tokens': 12, 'total_tokens': 289, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
from langchain_core.runnables import RunnableParallel

# Step 1a: Extract topic
prompt_topic = ChatPromptTemplate.from_template("What is the topic of: {input}?")
chain_topic = prompt_topic | llm

# Step 1b: Extract sentiment
prompt_sentiment = ChatPromptTemplate.from_template(
    "What is the sentiment of: {input}?"
)
chain_sentiment = prompt_sentiment | llm

# Step 2: Summarize using topic and sentiment
prompt_summary = ChatPromptTemplate.from_template(
    "Summarize this text: {original_input}\nFocusing on the topic: {topic}\nAdopt a {sentiment} tone."
)
chain_summary = prompt_summary | llm

# Combine using RunnableParallel to create a state dictionary
state_creation = RunnableParallel(
    topic=chain_topic,
    sentiment=chain_sentiment,
    original_input=RunnablePassthrough(),  # Pass the original input through
)

full_chain = state_creation | chain_summary

In [40]:
# Example invocation:
input_text = "The new product launch was a huge success, exceeding all expectations."
result = full_chain.invoke({"input": input_text})

pprint(result)

AIMessage(content='The text describes a successful product launch that exceeded expectations, with a positive sentiment.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 578, 'total_tokens': 594, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrELsOoyMWgMENEEWUMtGop2tywAu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--fa33857f-def0-4b9b-8400-23d6e0071b61-0', usage_metadata={'input_tokens': 578, 'output_tokens': 16, 'total_tokens': 594, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


In [41]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [42]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [46]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

# llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.invoke({"email_conversation": email_conversation})

# output = stream_response(answer, return_output=True)

In [47]:
print(answer.content)

중요 내용 요약:

- 발신자: 김철수 (바이크코퍼레이션 상무)
- 수신자: 이은채 (Teddy International)
- 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
- 요청 사항: ZENESIS 모델에 대한 상세 브로슈어 (기술 사양, 배터리 성능, 디자인 정보)
- 미팅 제안: 다음 주 화요일(1월 15일) 오전 10시, 귀사 사무실에서 만남 제안
- 목적: 협력 가능성 논의 및 유통 전략, 마케팅 계획 구체화


In [48]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# instruction 을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [49]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [ ]:
# chain 을 생성합니다.
chain = prompt | llm

# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
pprint(response.content)

AIMessage(content='```json\n{\n  "person": "김철수",\n  "email": "chulsoo.kim@bikecorporation.me",\n  "subject": "\\"ZENESIS\\" 자전거 유통 협력 및 미팅 일정 제안",\n  "summary": "김철수 상무가 이은채 대리님에게 바이크코퍼레이션의 자전거 \'ZENESIS\'에 대한 브로슈어 요청과 협력 논의를 위한 미팅 제안을 보냈습니다.",\n  "date": "1월 15일 오전 10시"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 601, 'total_tokens': 722, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BrEsGNPPhDUMDtdqe5owNXsCmQFnc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--afe5d0b4-73b9-478e-a021-2cd0e4f61d2b-0', usage_metadata={'input_tokens': 601, 'output_tokens': 121, 'total_tokens': 722, 'input_token_details': {'audio': 0, 'cache

In [52]:
# 결과는 JSON 형태로 출력됩니다.
pprint(response.content)

('```json\n'
 '{\n'
 '  "person": "김철수",\n'
 '  "email": "chulsoo.kim@bikecorporation.me",\n'
 '  "subject": "\\"ZENESIS\\" 자전거 유통 협력 및 미팅 일정 제안",\n'
 '  "summary": "김철수 상무가 이은채 대리님에게 바이크코퍼레이션의 자전거 \'ZENESIS\'에 대한 브로슈어 요청과 협력 '
 '논의를 위한 미팅 제안을 보냈습니다.",\n'
 '  "date": "1월 15일 오전 10시"\n'
 '}\n'
 '```')


In [55]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(response.content)
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary="김철수 상무가 이은채 대리님에게 바이크코퍼레이션의 자전거 'ZENESIS'에 대한 브로슈어 요청과 협력 논의를 위한 미팅 제안을 보냈습니다." date='1월 15일 오전 10시'
